## AS02: Representação Textual

**Aluno: Vinícius Henrique Giovanini**  
**Matricula: 692225**  
**Professor: Wladmir Cardoso Brandão**  

In [1]:
from sklearn.datasets import fetch_20newsgroups
import pprint as pp
import re
import numpy
import json

In [2]:
newsgroup_train = fetch_20newsgroups(subset='train', shuffle=False, random_state=42)
pp.pprint(list(newsgroup_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [3]:
len(list(newsgroup_train.data))

11314

### Funções de Ajuda

In [12]:
def salvar_numpy_array_para_txt(nome_arquivo, array):
    numpy.savetxt(nome_arquivo, array, fmt='%s')

def ler_txt_para_numpy_array(nome_arquivo):
    lista = []

    with open(nome_arquivo, 'r') as arquivo:
        for linha in arquivo:
            lista.append(linha.strip())

    return numpy.array(lista)
  
def salvar_dict_arq(nome_arquivo, dict):
    with open(nome_arquivo, "w") as arquivo:
        for chave, valor in dict.items():
            linha = json.dumps({chave: valor.tolist()}) + '\n'
            arquivo.write(linha)
            
def carregar_dict_arq(nome_arquivo):
    dict_carregado = {}
    with open(nome_arquivo, "r") as arquivo:
        for linha in arquivo:
            obj_json = json.loads(linha)
            chave, valor = list(obj_json.items())[0]
            valor = numpy.array(valor)
            dict_carregado[chave] = valor
    return dict_carregado

In [5]:
def get_tokens(txt):
  tokens = re.sub("[^\w]", " ", txt).split()
  tokens = [w.lower() for w in tokens]
  return tokens

# Pre-processamento
Removendo caracteres especiais 

In [6]:
data_list = list(newsgroup_train.data)

In [7]:
def clean_txts(txt):
  tokens = re.sub("[^\w]", " ", txt).split()
  clean_txt = [w.lower() for w in tokens]
  return " ".join(clean_txt)

# Limpando o dataset inteiro removendo caracteres especiais e colocand to lower case
data_list_cleaned = []

for line in data_list:
 data_list_cleaned.append(clean_txts(line))

In [8]:
len(data_list), len(data_list_cleaned)

(11314, 11314)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
X_counts = vect.fit_transform(data_list_cleaned)
vocabulario = vect.get_feature_names_out()

In [10]:
len(vocabulario)

130107

In [14]:
salvar_numpy_array_para_txt("../saidas/AS02/data_list_processada.txt", vocabulario)

# One-Hot Encoding

**OBS**: Professor rodei criei o vocabulario e rodei o metodo para gerar o one hot enconding para cada frase, porem a celula passou 83m rodando e não terminou de fazer de todas as frases, pois tem 11k de frases com cerca de 100k de palavras. Por conta disso rodei para poucas frases (50 frases), deixei o codigo comentado pois a celula demora 15 minutos para rodar a celula, na entrega vou fazer ele chamar o txt para não precisar rodar a celula

In [35]:
def embed_frase(frase):
    palavras = frase.split()
    
    embedding_frase = numpy.zeros(len(vocabulario))
    
    for palavra in palavras:
        if palavra in vocabulario:
            indice = numpy.where(vocabulario == palavra)[0][0]
            embedding_frase[indice] = 1
    
    return embedding_frase


one_hot_encoding = {}

def hot_encoding(tam):
  for i, line in enumerate(data_list_cleaned):
      one_hot_encoding[line] = embed_frase(line)
      
      if i == tam:
        break
      i += 1

In [ ]:
# hot_encoding(50)

In [41]:
# salvar_dict_arq("../saidas/AS02/20News_01.txt", one_hot_encoding)

In [13]:
one_hot_encoding = carregar_dict_arq("../saidas/AS02/20News_01.txt")

In [21]:
one_hot_encoding

{'from cubbie garnet berkeley edu subject re cubs behind marlins how article i d agate 1pt592 f9a organization university of california berkeley lines 12 nntp posting host garnet berkeley edu gajarsky pilot njin net writes morgan and guzman will have era s 1 run higher than last year and the cubs will be idiots and not pitch harkey as much as hibbard castillo won t be good i think he s a stud pitcher this season so far morgan and guzman helped to lead the cubs at top in era even better than the rotation at atlanta cubs era at 0 056 while braves at 0 059 we know it is early in the season we cubs fans have learned how to enjoy the short triumph while it is still there': array([0., 0., 0., ..., 0., 0., 0.]),
 'from gnelson pion rutgers edu gregory nelson subject thanks apple free ethernet on my c610 article i d pion apr 6 12 05 34 1993 11732 organization rutgers univ new brunswick n j lines 26 well i just got my centris 610 yesterday it took just over two weeks from placing the order the 

# Count Vectors

In [25]:
one_hot_encoding_count_vectors = {}

def count_vectors():
  for key,value in one_hot_encoding.items():
    words = get_tokens(key)
    bag_vector = numpy.zeros(len(vocabulario))
    for w in words:
      for i,word in enumerate(vocabulario):
        if word == w:
          bag_vector[i] += 1
    one_hot_encoding_count_vectors[key] = numpy.array(bag_vector)
    
    
one_hot_encoding_count_vectors